In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020'])

In [3]:
file = FILES_TO_ANALYZE['Winterthur 2017']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='KSW', year='2017', sheets=['Änderungen _Winterthur_2017'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 218 cases for KSW 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [5]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,20066273027,D1B4E3C92520FC01,M,35,4,3,M60B,C61,M100,C787,<NA>,<NA>,<NA>
1,20093870027,13320B741B5845D4,M,84,17,3,I46C,S7210,M200,T840,<NA>,81525b:R:20170501,81525B:R:20170501
2,20158223027,BA43D16D9B8B472A,M,75,11,3,H01B,C787,M200,D684,<NA>,"502a27::20170803,009a13::20170803,99b711::2017...","502A27::20170803,009A13::20170803,99B711::2017..."
3,20219240027,7D29F200ABFD59CD,M,87,14,3,I46C,S7201,M100,B370,<NA>,<NA>,<NA>
4,20237303027,2BDCB561504F6656,W,83,7,3,I46C,S7201,M200,N182,<NA>,<NA>,<NA>


# match to the database


In [6]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['20066273027', '20093870027', '20158223027', '20219240027',
       '20237303027', '20520053027', '20521883027', '20595506027',
       '20651153027', '20652950027', '20715770027', '20027270027',
       '20184413027', '20256623027', '20330843027', '20418443027',
       '20536136027', '20603690027', '20603840027', '20722103027',
       '20728766027', '20801606027', '20049053027', '20149040027',
       '20270936027', '20353373027', '20359340027', '20492120027',
       '20498453027', '20500583027', '20113853027', '20418260027',
       '20712080027', '20830463027', '20971223027', '21008423027',
       '20549333027', '20560670027', '20574206027', '20629073027',
       '20716973027', '20863823027', '20906963027', '20912420027',
       '20919833027', '20975933027', '20992130027', '20999783027',
       '21034103027', '21050840027', '21121763027', '21213383027',
       '21246380027', '21261323027', '21768383027', '21976340027',
       '21977336027', '21086843027', '21089240027', '211172300

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,1.127,105733,13,20027270027,52C61126FF3E328E,49,0,M,18,4,0,0,0,2017-05-02,01,2017-05-20,00,G07Z,G07,3
1,0.759,97561,13,20049053027,5F28DF79C45509D6,57,0,W,14,3,0,0,0,2017-08-14,01,2017-08-28,00,H62B,H62,3
2,0.506,110807,13,20056370027,B83FE2382CB15363,89,0,W,6,3,0,0,0,2017-05-09,01,2017-05-15,00,F72B,F72,2
3,0.668,93530,13,20066273027,D1B4E3C92520FC01,35,0,M,4,3,0,0,0,2017-10-14,01,2017-10-18,00,M60B,M60,3
4,1.677,97960,13,20093870027,13320B741B5845D4,84,0,M,17,4,0,0,0,2017-04-29,01,2017-05-16,00,I46C,I46,3


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 218 out of 218 revised cases from DtoD that are matched with the database for KSW 2017


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match